
# Getting Started with PSBlueSky

An overview of Jeff Hick's cool PSBlueSky module, from installation to posting from the Command Line. (plus everything is cooler in the CLI ;) )

## How to Install

Can grab it from the PowerShell Gallery via Import-module or Install-PSFModule (and for the PSFRamework.nuget users: Install-PSFModule)

In [ ]:
Install-Module -Name PSBluesky -Scope CurrentUser -Force -AllowClobber

In [ ]:
Install-PSResource -Name PSBluesky -Repository PSGallery -TrustRepository

In [ ]:
Install-PSFModule -Name PSBlueSky

## BlueSky Credentials

Jeff wisely recomends you store your account credentials you need to get from  https://bsky.app/ and store in a vault

• Microsoft.PowerShell.SecretManagement - https://github.com/powershell/secretmanagement
• Microsoft.PowerShell.SecretStore - https://github.com/powershell/secretstore

I have a keepass one setup, can find out more in my blog (advise following Jeff's suggestion)

Powershell Profiles Keepass And Psbluesky | PowerShell Young Team
https://powershellyoungteam.github.io/2024/11/22/PowerShell-Profiles-KeePass-and-PSBlueSky.html

NOTE: Username is case sensitive too!, also don’t be like me and put the @ sign before the username, so like PoshYoungTeam.bsky.social, NOT @PoshYoungTeam.bsky.social

if you use 2FA you will need to create an App password, there is detailed instructions in the help pdf.

In [ ]:
import-module PSBlueSky
Import-Module SecretManagement.KeePass

$BskyCreds = Get-Secret -Vault PowerShell -Name BlueSky

# if you don't have a vault but want to quickly follow on, you can use the following
$BskyCreds = Get-Credential -Message "Enter your BlueSky credentials"

## Help!

PSBlueSky has some Gucci built in help! check out these Cmdletts as well as Github help and built in Help (get-Help)

In [ ]:
Open-BskyHelp # which will open a PDF version of this document in your default PDF viewer

Get-BskyModuleInfo # which will show you the module commands and has clickable links to online help(module's github page)

## Start a session

you need to use the creds in your vault!

you can create a session, and if you get a timeout error, you can refresh the session.


In [ ]:
Start-BskySession -credential $cred

Get-BskySession # confirm that you are connected to the session

Get-BSkySession | Update-BskySession # to update the session


## Profile

so I have added a few lines into my Profile so I get prompted for my vault password, and a BlueSky session gets started etc.. (can even retrieve stuff like notifications or feed etc...)

In [ ]:
import-module PSBlueSky
Import-Module SecretManagement.KeePass

$BskyCreds = Get-Secret -Vault PowerShell -Name BlueSky

Start-BSkySession -Credential $BskyCreds
Get-BskyNotification | Where-Object { $_.Date -gt (Get-Date).AddDays(-1) } | Sort-Object Date

## Sending your first Skeet

very easy, and as alwys with powershell, more than one way!

In [ ]:
# Posting! this was my 2nd post..

New-BskyPost -message "It works!! :)" -imagepath C:\users\Wighty\Pictures\BskyPic.PNG -ImageAlt "pic of my first PSBlueSky post" -Verbose -WhatIf

# pass as an object

$param = @{
    Message = "It works!! :)"
    ImagePath = "C:\users\Wighty\Pictures\BskyPic.PNG "
    ImageAlt = "pic of my first PSBlueSky post"
    Verbose = $true
}

New-BskyPost @param -WhatIf

# with a URL, make sure URL is surrounded by Whitespace

New-BskyPost "Cool #PowerShell tip popped up in my terminal today from Daniel Schroeder ( @deadlydog.bsky.social ). In short, watch this Github repo to stay up to date with known PwSh security vulnerabilities and breaking changes:   [PowerShell Annoucements] (https://github.com/PowerShell/Announcements) " -Verbose

# multiline post, save text as string and pass to New-BskyPost

$multiline = @"
I'm really enjoying the #PSBlueSky module. It's a great way to post to Bluesky from #PowerShell. I can't wait to see what other features are added in the future.

#PowerShell
#PSBlueSky
"@

New-BskyPost $multiline -Verbose -WhatIf

## Feeds, Timelines, Notifications and reposts

fairly straightforward, although you may want to filter somehow (if you don't want to fill you terminal)

In [ ]:
# Feeds, Timelines Notifications and reposts

Get-BskyFeed -Verbose # get your feed (good to filter down by date or limit)

#jeff hicks tip for tips!!

Get-BskyFeed -UserName steviecoaster.dev | Where-Object text -match 'Pwsh Tip of the day' -ov w | Sort-Object -Property Date -Descending | Format-List -Property text -GroupBy date

Get-BskyFeed | Where-Object { $_.Text -match "PowerShell|Pwsh"  } # filter by atext, keywords

Get-BskyFeed | where-object { $_.Author -eq $BskyCreds.Username} | Sort-Object Liked # get my most liked skeets

Get-BskyTimeline -Verbose # get your timeline (good to filter down by time or limit)

#get timeline and filter by skeets containing the word "PowerShell" or "Pwsh" or by date
Get-BskyTimeline | Where-Object { $_.Text -match "PowerShell|Pwsh" } # filter by text
Get-BskyTimeline | Where-Object { $_.Date -gt (Get-Date).AddDays(-1) } # filter by date (yesterday)

Get-BskyNotification -Verbose # get your notifications with clickable links to the post

# filter by date (yesterday)
Get-BskyNotification | Where-Object { $_.Date -gt (Get-Date).AddDays(-1) } # filter by date (yesterday)

# reposting, need non -default properties URI and CID

Get-BskyTimeline | Where-Object { $_.Text -match "PowerShell|Pwsh" } | Select-Object Author,Date,Text,URI, CID | Publish-BskyPost -whatif

# countdown to PowerShell Wednesday
# Calculate time until 1900 GMT and create a requote (from Copilot)
$timeToGo = [math]::Ceiling(((Get-Date "19:00:00Z").ToUniversalTime() - (Get-Date).ToUniversalTime()).TotalMinutes)
Get-BskyFeed | Where-Object { $_.Text -match "#PowerShellWednesday"  }   | Select-Object -First 1 | Select-Object Author,Date,Text,URI,CID | Publish-BskyPost -Quote "only $timeToGo minutes until #PowerShellWednesday" -WhatIf

# reposting, need non -default properties URI and CID
Get-BskyFeed | Where-Object { $_.Text -match "#PowerShellWednesday"  }   | gn -whatif



## Users

How to interact with user via PSBlueSKy (also keep an eye on your follower count)

In [ ]:
# User Stuff

Find-BskyUser -UserName "jsnover.com"  # find anyone with PowerShell in their username and Description

Find-BskyUser -UserName "powershell" | Select-Object username # use this to get a list of usernames for demo purposes :)

Get-BskyProfile # Get your profile/bio or pass another users username

Get-BskyFollowers # get your followers

# see how big the Young Team is

$YoungTeamers = Get-BskyFollowers
Write-Output "You have $($YoungTeamers.Count) muckers in the Young Team"

# let's make a list of the Young Team
Write-Output "The PowerShell Young Team:"; $YoungTeamers | Select-Object username | Sort-Object username | Format-Table -AutoSize

#Let's visit the Young Team

#get the path to my MSedge.exe
$EdgePath = 'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'
$YoungTeamers | ForEach-Object { Start-Process $EdgePath $_.URL }

Get-BskyFollowing # get who you are following , could be useful for a follow list

# see who you are following
Get-BskyFollowing | Select-Object username | Sort-Object username | Format-Table -AutoSize

# Following, Unfollowing and blocking badies!

New-BskyFollow "PoshYoungTeam.bsky.social" -WhatIf

#or get pipy with it

Get-BskyProfile "PoshYoungTeam.bsky.social"  | New-BskyFollow -passthru -whatif

#remove people from your follow list

Remove-BskyFollow "PoshYoungTeam.bsky.social" -WhatIf

#or get pipy with it

Get-BskyProfile "PoshYoungTeam.bsky.social" | Remove-BskyFollow -passthru -WhatIf

#blocking

Get-BskyBlockedUser # get who you have blocked

Block-BskyUser -UserName "baduser" -WhatIf

Unblock-BskyUser -UserName "baduser" -WhatIf


## Resources

• GitHub - PSBlueSky - https://github.com/jdhitsolutions/PSBluesky
Bluesky API Documentation - [HTTP Reference | Bluesky  ](https://docs.bsky.app/docs/category/http-reference)

https://github.com/jdhitsolutions/PSBluesky/discussions - checking this out for other uses!!!

https://github.com/jdhitsolutions/PSBluesky/discussions/35 - mdgrs - taskbar notifications!!!

https://github.com/jdhitsolutions/PSBluesky/discussions/28 - tip of the day

I've shared the ways I'm using the PSBlueSky #PowerShell module in the repo's Discussion section. How are you using
the module? jeffhicks bluesky https://bsky.app/profile/did:plc:ohgsqpfsbocaaxusxqlgfvd7/post/3li3j3yqxbh23

and see what people say here:

Jeff Brown Tech @jeffbrowntech.bsky.social

Using it in Azure Functions to call it from a Logic App for posting new blogs and videos to social.

